<a href="https://colab.research.google.com/github/topcat101-cmd/asdasdasd/blob/main/notes%20-%20linear%20regression%20-.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np #mulitdemanal ray, allows you fastly perform diffrent scnarios
import pandas as pd #data analitics tool
import matplotlib.pyplot as plt #
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf
#------------------------------
# !pip install -q sklearn
# print(tf.version)
# t = tf.zeros([5,5,5,5])
# t = tf.reshape(t, [125, -1])
# print(t)



In [ ]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data 
# this would be to train the model with>>>
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
#This would be to test the model with>>>
#print(dftrain.head()) #prints out the data frame
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')
#print(dftrain.loc[0], y_train.loc[0]) #prints out the data frame ----- #loc allows you to find a specific row in your dataframe, giving the index

Categorical_columns = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']

numeric_columns = ['age', 'fare']

feature_columns = []
for feature_name in Categorical_columns:
  vocabulary = dftrain[feature_name].unique()#gets a list of all the unoque values from a given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))#Puts all of this data into the feature columns list

for feature_name in numeric_columns:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))#Puts all of this data into the feature columns list

print(feature_columns)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=1000, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns) #this would give the feature columns from above, which would be passed to a linearobject classifier, to then estimator from tensorflow

linear_est.train(train_input_fn) #train the model
result = linear_est.evaluate(eval_input_fn) #get model metrics/ stats by testing on testing data

clear_output() #clears console output
print(result['accuracy']) #the result variable is simply a dict of stats about model
print(result)

0.78409094
{'accuracy': 0.78409094, 'accuracy_baseline': 0.625, 'auc': 0.8386899, 'auc_precision_recall': 0.7776822, 'average_loss': 0.47217602, 'label/mean': 0.375, 'loss': 0.45205894, 'precision': 0.6981132, 'prediction/mean': 0.40743992, 'recall': 0.74747473, 'global_step': 20000}


In [ ]:
result = list(linear_est.predict(eval_input_fn))
clear_output()
print(dfeval.loc[2])
print(y_eval.loc[2])
print(result[2]['probabilities'][1])
# making prediction and to then look at the predictions  #Linear regression

sex                        female
age                            58
n_siblings_spouses              0
parch                           0
fare                        26.55
class                       First
deck                            C
embark_town           Southampton
alone                           y
Name: 2, dtype: object
1
0.7625139
